In [247]:
# import of the needed packages
import pandas as pd
import numpy as np
import re
import math
import csv

file = 'cumulated_data_bl1_v3_with_mw_and_form.csv'

In [248]:
def load_data(file):
    data = pd.read_csv(file, delimiter=',')
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0
    return data

In [249]:
def get_all_teams(data):
    teams = []
    for x in data['HomeTeam']:
        if x not in teams:
            teams.append(x)
    return teams

In [250]:
def create_elo_dictionary(teams):
    teams_elo_avstaerke = {}
    for x in teams:
        team_stats = {x: [1000, 1000, 1.34, -1.34]}
        teams_elo_avstaerke.update(team_stats)
    return teams_elo_avstaerke

In [251]:
def diffelo(homeelo, awayelo):
    diffelo = homeelo - awayelo
    return diffelo

In [252]:
def elogain_2(z, data, hometeam, awayteam, teams_elo_avstaerke):
    homeelo = teams_elo_avstaerke[hometeam][0]
    awayelo = teams_elo_avstaerke[awayteam][0]

    trueoutcome = outcometrue_2(z, data)
    expectedoutcome = outcomeexpected_2(z, homeelo, awayelo)
    ofvictorymargin = marginofvictory_2(z, data, homeelo, awayelo)

    elogain = 20 * (trueoutcome - expectedoutcome) * ofvictorymargin
    elohome = homeelo + elogain
    eloaway = awayelo - elogain

    teams_elo_avstaerke[hometeam][0] = elohome
    teams_elo_avstaerke[awayteam][1] = eloaway

    return elohome, eloaway, homeelo, awayelo

In [253]:
def marginofvictory_2(z, data, homeelo, awayelo):
    elodiff = diffelo(homeelo, awayelo)
    diffgoal = data.FTHG[z] - data.FTAG[z]

    if diffgoal <= 1:
        marginofvictory = 1
    else:
        marginofvictory = (math.log2(1.7 * diffgoal) * 2) / (2 +
                                                             0.001 * elodiff)
    return marginofvictory

In [254]:
def outcometrue_2(z, data):
    if data.FTR[z] == "H":
        outcometrue = 1
    elif data.FTR[z] == "D":
        outcometrue = 0.5
    else:
        outcometrue = 0
    return outcometrue

In [255]:
def outcomeexpected_2(z, homeelo, awayelo):
    elodiff = diffelo(homeelo, awayelo)
    outcomeexpected = 1 / (1 + 10**(-elodiff / 400))
    return outcomeexpected

In [256]:
def home_attack(z, data, alpha, ratio, hometeam, awayteam,
                teams_elo_avstaerke):
    goalsshot = data.FTHG[z]
    attack_old = teams_elo_avstaerke[hometeam][2]
    otherteamdefend = teams_elo_avstaerke[awayteam][3]
    
    homeattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[hometeam][2] = homeattack
    return homeattack, attack_old

In [257]:
def home_defend(z, alpha, ratio, data, hometeam, awayteam,
                teams_elo_avstaerke):
    goalsgot = data.FTAG[z]
    defend_old = teams_elo_avstaerke[hometeam][3]
    otherteamattack = teams_elo_avstaerke[awayteam][2]
    
    homedefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[hometeam][3] = homedefend
    return homedefend, defend_old

In [258]:
def away_attack(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    goalsshot = data.FTAG[z]
    attack_old = teams_elo_avstaerke[awayteam][2]
    otherteamdefend = teams_elo_avstaerke[hometeam][3]
    
    awayattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[awayteam][2] = awayattack
    return awayattack, attack_old

In [259]:
def away_defend(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    goalsgot = data.FTHG[z]
    defend_old = teams_elo_avstaerke[awayteam][3]
    otherteamattack = teams_elo_avstaerke[hometeam][2]
    awaydefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[awayteam][3] = awaydefend
    return awaydefend, defend_old

In [260]:
def add_new_columns1(data):
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0

In [261]:
def add_new_columns2(data):
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0

In [262]:
def calculate_elo_attack_defense(data, teams_elo_avstaerke):
    z = 0
    while z < len(data):
        alpha = 0.5
        ratio = 0.75
        hometeam = data.HomeTeam[z]
        awayteam = data.AwayTeam[z]

        try:

            homeelo, awayelo, homeeloold, awayeloold = elogain_2(
            z, data, hometeam, awayteam, teams_elo_avstaerke)
            homeattack1, homeattackold = home_attack(z, data, alpha, ratio,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            homedefend1, homedefendold = home_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            awayattack1, awayattackold = away_attack(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            awaydefend1, awaydefendold = away_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
        except:
            print(f"Can't find data for team: {hometeam, awayteam}")
            break
        data['HomeElo'][z] = homeelo
        data['AwayElo'][z] = awayelo
        data['HomeAttack'][z] = homeattack1
        data['HomeDefend'][z] = homedefend1
        data['AwayAttack'][z] = awayattack1
        data['AwayDefend'][z] = awaydefend1
        data['HomeEloOld'][z] = homeeloold
        data['AwayEloOld'][z] = awayeloold
        data['HomeAttackOld'][z] = homeattackold
        data['HomeDefendOld'][z] = homedefendold
        data['AwayAttackOld'][z] = awayattackold
        data['AwayDefendOld'][z] = awaydefendold
        z+=1
    return data
    # return homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold


In [263]:
def add_elo_attack_defense(homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold):
    data['HomeElo'][z] = homeelo
    data['AwayElo'][z] = awayelo
    data['HomeAttack'][z] = homeattack1
    data['HomeDefend'][z] = homedefend1
    data['AwayAttack'][z] = awayattack1
    data['AwayDefend'][z] = awaydefend1
    data['HomeEloOld'][z] = homeeloold
    data['AwayEloOld'][z] = awayeloold
    data['HomeAttackOld'][z] = homeattackold
    data['HomeDefendOld'][z] = homedefendold
    data['AwayAttackOld'][z] = awayattackold
    data['AwayDefendOld'][z] = awaydefendold


In [264]:
def add_differences(data):
    data['DiffEloOld'] = data['HomeEloOld'] - data['AwayEloOld']
    data['DiffAttackOld'] = data['HomeAttackOld'] - data['AwayAttackOld']
    data['DiffDefendOld'] = data['HomeDefendOld'] - data['AwayDefendOld']
    data['DiffElo'] = data['HomeElo'] - data['AwayElo']
    data['DiffAttack'] = data['HomeAttack'] - data['AwayAttack']
    data['DiffDefend'] = data['HomeDefend'] - data['AwayDefend']
    return data

In [265]:
def add_qoutient(data):
    data['QuotEloOld'] = data['HomeEloOld'] / data['AwayEloOld']
    data['QuotAttackOld'] = data['HomeAttackOld'] / data['AwayAttackOld']
    data['QuotDefendOld'] = data['HomeDefendOld'] / data['AwayDefendOld']
    data['QuotElo'] = data['HomeElo'] / data['AwayElo']
    data['QuotAttack'] = data['HomeAttack'] / data['AwayAttack']
    data['QuotDefend'] = data['HomeDefend'] / data['AwayDefend']
    return data

In [273]:
def direct_comparison(data):
    # get direct comparison of the two teams

    direct_comparison_hg = []
    direct_comparison_ag = []
    direct_comparison_hp = []
    direct_comparison_ap = []

    for counter, rows in data.iterrows():
        data_games_played_until_now = data.iloc[:counter]
        direct_comparison_teams_temp = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)&(data_games_played_until_now.AwayTeam == rows.AwayTeam)|
        (data_games_played_until_now.HomeTeam == rows.AwayTeam)&(data_games_played_until_now.AwayTeam == rows.HomeTeam)]
        if len(direct_comparison_teams_temp) >= 3:
            if len(direct_comparison_teams_temp) >= 5:
                # take only the last 5 matches
                direct_comparison_teams_temp = direct_comparison_teams_temp.iloc[-5:]
                #direct_comparison_teams_temp.iloc[-5:].value_counts(["HomeTeam", "FTAG"])
                number_of_games = 5
            else:
                number_of_games = len(direct_comparison_teams_temp)

            # get average number of goals, that the hometeam has scored in the last 5 direct matches
            home_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.HomeTeam].FTAG.sum()) / number_of_games

            # get average number of goals, that the awayteam has scored in the last 5 direct matches
            away_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.AwayTeam].FTAG.sum()) / number_of_games

        else:
            # if the direct comparison is not possible yet (because both teams haven't played against each other) the scored goals of the last 10 matches would be measured to avoid null values
            goals_scored_ht = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)|(data_games_played_until_now.AwayTeam == rows.HomeTeam)].iloc[-10:]
            goals_scored_at = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.AwayTeam)|(data_games_played_until_now.AwayTeam == rows.AwayTeam)].iloc[-10:]

            if len(goals_scored_ht) >= 10:
                home_goals = (goals_scored_ht[goals_scored_ht.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                    goals_scored_ht[goals_scored_ht.AwayTeam == rows.HomeTeam].FTAG.sum()) / 10
                away_goals = (goals_scored_at[goals_scored_at.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                    goals_scored_at[goals_scored_at.AwayTeam == rows.AwayTeam].FTAG.sum()) / 10
            else:
                home_goals = None
                away_goals = None

        direct_comparison_hg.append(home_goals)
        direct_comparison_ag.append(away_goals)

    print(direct_comparison_hg)
    print(direct_comparison_ag)
    print(direct_comparison_hp)
    print(direct_comparison_ap)

    data['DirectComparisonHG'] = direct_comparison_hg
    data['DirectComparisonAG'] = direct_comparison_ag
    data['DirectComparisonGoalDiff'] = data[
            'DirectComparisonHG'] - data['DirectComparisonAG']
    data['DirectComparisonGoalQuot'] = data['DirectComparisonHG'] / data[
    'DirectComparisonAG']
    return data


In [269]:
def select_necessary_columns(data):
    data_reduced = data[[
        "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "HS", "AS",
        "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "PointsLast3MatchesHT",
        "PointsLast3MatchesAT", "PointsLast5MatchesHT", "PointsLast5MatchesAT",
        "PointsLast10MatchesHT", "PointsLast10MatchesAT", "PointsLastAllMatchesHT",
        "PointsLastAllMatchesAT", "PDiff3Matches", "PQuot3Matches", "PDiff5Matches",
        "PQuot5Matches", "PDiff10Matches", "PQuot10Matches", "PDiffAllMatches",
        "PQuotAllMatches", "MarketValueHT", "MarketValueAT", "MarketValueDiff",
        "MarketValueQuot", "HomeElo", "AwayElo", "HomeAttack", "HomeDefend", "AwayAttack",
        "AwayDefend", "HomeEloOld", "AwayEloOld", "HomeAttackOld", "HomeDefendOld",
        "AwayAttackOld", "AwayDefendOld", "DiffEloOld", "DiffAttackOld", "DiffDefendOld",
        "DiffElo", "DiffAttack", "DiffDefend", "QuotEloOld", "QuotAttackOld",
        "QuotDefendOld", "QuotElo", "QuotAttack", "QuotDefend", "DirectComparisonHG",
        "DirectComparisonAG", "DirectComparisonGoalDiff", "DirectComparisonGoalQuot"
    ]]
    return data_reduced


In [274]:
def run_all(file):
    data = load_data(file)
    teams = get_all_teams(data)
    teams_elo_avstaerke = create_elo_dictionary(teams)
    data = calculate_elo_attack_defense(data, teams_elo_avstaerke)
    data = add_differences(data)
    data = add_qoutient(data)
    data = direct_comparison(data)
    data_reduced = select_necessary_columns(data)
    return data_reduced


data = run_all(file)

C:\Users\I539022\AppData\Local\Temp/ipykernel_38456/127603076.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HomeElo'][z] = homeelo
C:\Users\I539022\AppData\Local\Temp/ipykernel_38456/127603076.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['AwayElo'][z] = awayelo
C:\Users\I539022\AppData\Local\Temp/ipykernel_38456/127603076.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HomeAttack'][z] = homeattack1


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1.3, 0.9, 1.2, 2.1, 1.6, 1.7, 0.9, 1.4, 1.0, 2.6, 1.2, 0.9, 1.8, 1.4, 1.4, 0.9, 1.7, 0.8, 1.1, 1.8, 1.5, 1.2, 1.5, 0.7, 1.0, 1.9, 0.9, 1.9, 1.4, 0.7, 1.0, 1.8, 2.7, 0.8, 1.6, 2.2, 0.6, 1.8, 1.7, 1.5, 1.2, 1.3, 2.8, 1.4, 1.3, 1.7, 1.7, 1.4, 0.8, 0.9, 0.6, 1.9, 1.9, 0.8, 1.4, 1.5, 1.4, 1.1, 1.7, 0.5, 1.1, 1.3, 3.3, 1.1, 1.3, 1.6, 0.9, 1.3, 1.6, 1.3, 1.5, 2.9, 2.0, 0.4, 1.5, 0.8, 0.7, 0.7, 2.0, 0.5, 2.1, 1.4, 1.6, 1.0, 0.9, 1.4, 1.6, 0.9, 1.2, 2.5, 1.6, 0.9,

In [ ]:
# betting_providers = [
#     'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'VCH',
#     'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'WHH', 'WHD', 'WHA', 'SBH', 'SBD',
#     'SBA', 'SJH', 'SJD', 'SJA', 'LBH', 'LBD', 'LBA', 'Bb1X2', 'BbMxH', 'BbAvH',
#     'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx > 2.5',' BbAv > 2.5', 'BbMx < 2.5',
#     'BbAv < 2.5', 'BbAH, BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'GPHTH', 'GPHTA',
#     'GPATH', 'GPATA', 'GPHT', 'GPAT'
# ]
# for x in betting_providers:
#     try:
#         del data[x]
#     except:
#         print(x)

In [275]:
data.tail(50)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,QuotEloOld,QuotAttackOld,QuotDefendOld,QuotElo,QuotAttack,QuotDefend,DirectComparisonHG,DirectComparisonAG,DirectComparisonGoalDiff,DirectComparisonGoalQuot
4846,09/04/2022,Wolfsburg,Bielefeld,4,0,H,2,0,H,11,...,1.320532,NaN,2.000000,1.326425,inf,0.50,2.800000,0.800000,2.000000,3.500000
4847,09/04/2022,Hertha,Union Berlin,1,4,A,0,1,A,12,...,1.026574,inf,2.000000,1.013777,0.500000,3.00,1.600000,1.000000,0.600000,1.600000
4848,10/04/2022,Bochum,Leverkusen,0,0,D,0,0,D,11,...,0.725159,1.000000,-inf,0.733404,0.000000,NaN,1.000000,1.600000,-0.600000,0.625000
4849,10/04/2022,Ein Frankfurt,Freiburg,1,2,A,0,1,A,17,...,0.993026,0.000000,-0.000000,0.983449,0.000000,-0.00,2.000000,1.400000,0.600000,1.428571
4850,10/04/2022,RB Leipzig,Hoffenheim,3,0,H,3,0,H,11,...,1.033970,inf,-0.000000,1.054155,inf,-0.00,1.200000,0.600000,0.600000,2.000000
4851,16/04/2022,Augsburg,Hertha,0,1,A,0,0,D,13,...,1.019253,1.000000,-0.000000,1.006753,0.000000,-0.00,1.200000,1.600000,-0.400000,0.750000
4852,16/04/2022,Dortmund,Wolfsburg,6,1,H,5,0,H,12,...,1.122210,0.500000,1.000000,1.137156,3.000000,-0.00,2.400000,0.200000,2.200000,12.000000
4853,16/04/2022,Freiburg,Bochum,3,0,H,2,0,H,14,...,1.349747,inf,-inf,1.353154,inf,1.00,1.333333,1.333333,0.000000,1.000000
4854,16/04/2022,Mainz,Stuttgart,0,0,D,0,0,D,9,...,1.047921,inf,2.000000,1.045559,NaN,1.00,1.200000,2.000000,-0.800000,0.600000
4855,16/04/2022,M'gladbach,FC Koln,1,3,A,0,3,A,18,...,1.056027,1.000000,-0.000000,1.041573,0.500000,1.00,1.600000,1.600000,0.000000,1.000000


In [272]:
data.to_csv(
    "./preprocessed_dataframe_with_elo_mw_form_3_v3.csv")
